In [3]:
import pandas as pd
import numpy as np 
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [4]:
#add torch reproducibility 

In [5]:
xiang_filtered = pd.read_csv("xiang_filtered.csv")
xiang_filtered_embeddings = torch.load("filtered_embeddings.pt")
#xiang_filtered_embeddings = xiang_filtered_embeddings["embeddings"]

#test1 xiang, test2 smash

In [6]:
xiang_filtered

,BGC code in MIBiG or name of product,Name of protein,Order of module,Annotation,Extender,Organism,Taxonomy,Sequence
0,actinopyrone,atpn1,M2,B1,MM,Streptomyces sp. SCSIO ZS0520,Actinomycetes,HTTLVTGGTGALGPHIARWLAEAGAERLVVTSRRGPLAEGMPELVA...
1,aetokthonostatin,aesJ,M4,A,M,Aetokthonos hydrillicola Thurmond2011,Cyanobacteriota,GVYLITGGLGNMGMVLSEFLASNYQAKLLIVSSRAFPKSSQWQEWL...
2,aetokthonostatin,aesB,M6,A1,M,Aetokthonos hydrillicola Thurmond2011,Cyanobacteriota,GVYLIFGGLGRVGLLLALDLAQRVKAKLVLVSRSELPPRSEWESWF...
3,aminoansamycin,aasA,M1,A,M,Streptomyces sp.,Actinomycetes,GTVLITGGSGGVGARIARRLARAGAGHLLLSSRRGPAAEGAAELVA...
4,Amphotericin,AF357202_AmphB1,M1,A2,MM,Streptomyces nodosus,Actinomycetes,GTTLVTGGSGTLAPGLARHLAAQGAEHLVLLSRRGADAPGAAELAA...
...,...,...,...,...,...,...,...,...
757,vancoresmycin,PKS4,M8,B,M,Amycolatopsis sp.,Actinomycetes,GGGLVPDAAAGARHRRGARARMLTSRRGTDAPGAAELVAELTAAGT...
758,vancoresmycin,PKS5,M9,A,M,Amycolatopsis sp.,Actinomycetes,GTVLVTGGTGALGSAVARWAAQAGAAHVILLSRSGPEAPGAAALLE...
759,venediol,vedC,M5,B1,MM,Streptomyces venezuelae,Actinomycetes,GTVLITGGTGSLGALFARHLVTRHGVRDLLLTSRRGDKADGARELG...
760,verticilactam,vtlP4,M6,B,M,Streptomyces spiroverticillatus,Actinomycetes,GTTLITGGTGALGAHLARSLARHGAAHLVLTSRRGSDAPGAAELAA...


In [7]:
import pandas as pd

kr_class_map = {
    'A':'A',
    'A1': 'A',
    'A2': 'A2',
    'B':'B',
    'B1': 'B',
    'B2': 'B2',
    'C':'C',
    'C1': 'C',
    'C2': 'C2'
}
xiang_filtered['Annotation'] = xiang_filtered['Annotation'].replace(kr_class_map)


In [8]:
#average pooling. [len_of_seq, 1, 1536]


xiang_filtered_embeddings = [x.mean(dim=1).squeeze(0) for x in xiang_filtered_embeddings]

In [9]:
print(xiang_filtered_embeddings[0].shape)

torch.Size([1536])


In [10]:
xiang_filtered_embeddings[1].shape

torch.Size([1536])

In [11]:
#adding onehot encoding
#xiang_filtered_embeddings = [torch.cat((xiang_filtered_embeddings[i],xiang_extenders_onehot[i]), dim=0) for i in range(len(xiang_filtered_embeddings))]
#xiang_filtered_embeddings = [torch.cat((xiang_filtered_embeddings[i],xiang_orders_onehot[i]), dim=0) for i in range(len(xiang_filtered_embeddings))]

In [12]:
xiang_filtered_embeddings[0]

tensor([ 18.4311,  95.8367,   9.3923,  ..., -42.4762, -42.7177, -35.4733])

In [13]:
#stacking
xiang_embeddings = torch.stack(xiang_filtered_embeddings)

In [14]:

#this is standarization but we should do it AFTER -- on training only. 
'''
# After stacking embeddings
print(f"Before normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")

# Standardize features
mu = xiang_embeddings.mean(0)
sigma = xiang_embeddings.std(0) + 1e-9
xiang_embeddings = (xiang_embeddings - mu) / sigma

print(f"After normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")
'''

'\n# After stacking embeddings\nprint(f"Before normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")\n\n# Standardize features\nmu = xiang_embeddings.mean(0)\nsigma = xiang_embeddings.std(0) + 1e-9\nxiang_embeddings = (xiang_embeddings - mu) / sigma\n\nprint(f"After normalization - mean: {xiang_embeddings.mean():.3f}, std: {xiang_embeddings.std():.3f}")\n'

In [15]:
#outputs need to be converted to numerical values.

annotations_unique = xiang_filtered["Annotation"].unique()
annotations_unique.sort() #sort alphabetically 

annotation_enumerated = {x: i for i, x in enumerate(annotations_unique)}
print(annotation_enumerated)

{'A': 0, 'A2': 1, 'B': 2, 'B2': 3, 'C': 4, 'C2': 5}


In [16]:
xiang_filtered["AnnotationEnumerated"] = xiang_filtered["Annotation"].map(annotation_enumerated)

In [17]:
print(xiang_filtered["AnnotationEnumerated"])

0      2
1      0
2      0
3      0
4      1
      ..
757    2
758    0
759    2
760    2
761    2
Name: AnnotationEnumerated, Length: 762, dtype: int64


In [18]:
xiang_filtered_np = xiang_filtered["AnnotationEnumerated"].to_list()

In [19]:
#xiang_filtered_tensor = torch.tensor(xiang_filtered_np)
#xiang_filtered_tensor = [torch.tensor(x, dtype=torch.long) for x in xiang_filtered_np]

xiang_filtered_tensor = torch.tensor(xiang_filtered_np, dtype=torch.long)

In [20]:
from sklearn.model_selection import train_test_split

x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor = train_test_split(
    xiang_embeddings,
    xiang_filtered_tensor,
    test_size = 0.2,
    random_state=1,
    stratify=xiang_filtered_tensor
)

mu, sigma = x_train_tensor.mean(0), x_train_tensor.std(0) + 1e-9
x_train_tensor = (x_train_tensor - mu) / sigma
x_test_tensor = (x_test_tensor - mu) / sigma

print("x train len")
print(len(x_train_tensor))
print("y train len")
print(len(y_train_tensor))

x train len
609
y train len
609


In [21]:
'''
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.20),
            nn.Linear(256,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.20),
'''

'\n            nn.BatchNorm1d(1024),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n            nn.Linear(1024, 512),\n            nn.BatchNorm1d(512),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n            nn.Linear(512, 256),\n            nn.BatchNorm1d(256),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n            nn.Linear(256,128),\n            nn.BatchNorm1d(128),\n            nn.ReLU(),\n            nn.Dropout(0.20),\n'

In [22]:
xiang_embeddings[0].shape[0]

1536

In [23]:

#add dropout_rate. ==
class kr_predict(nn.Module):
    def __init__(self):
        super(kr_predict, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(xiang_embeddings[0].shape[0], 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512,256),
            nn.ReLU(),
            nn.Dropout(0.25),
        )
        self.out = nn.Linear(256, 6)
    def forward(self, x):
        #x = x.view(x.size(0), -1) # flatten so we're removing 
        x = self.hidden(x)
        x = self.out(x)
        return x      



'''
class kr_predict(nn.Module):
    def __init__(self):
        super(kr_predict, self).__init__()
        self.hidden = nn.Sequential(
            nn.Linear(1536, 512),     # Slightly wider first layer
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(), 
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        self.out = nn.Linear(128, 9)
    def forward(self, x):
        #x = x.view(x.size(0), -1) # flatten so we're removing - modified so its done w/ squeeze
        x = self.hidden(x)
        x = self.out(x)
        return x 
'''

"\nclass kr_predict(nn.Module):\n    def __init__(self):\n        super(kr_predict, self).__init__()\n        self.hidden = nn.Sequential(\n            nn.Linear(1536, 512),     # Slightly wider first layer\n            nn.ReLU(),\n            nn.Dropout(0.3),\n            nn.Linear(512, 256),\n            nn.ReLU(), \n            nn.Dropout(0.3),\n            nn.Linear(256, 128),\n            nn.ReLU(),\n            nn.Dropout(0.3),\n        )\n        self.out = nn.Linear(128, 9)\n    def forward(self, x):\n        #x = x.view(x.size(0), -1) # flatten so we're removing - modified so its done w/ squeeze\n        x = self.hidden(x)\n        x = self.out(x)\n        return x \n"

In [24]:
#xiang_embeddings[0].shape[1]

In [25]:
model = kr_predict()

In [26]:
#apple silicon

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

if torch.backends.mps.is_available():
    print("MPS is available! Using Apple Silicon GPU.")
else:
    print("MPS is not available. CPU Fallback.")

MPS is available! Using Apple Silicon GPU.


In [27]:
adam = optim.Adam(model.parameters(), lr = .00001)
from collections import Counter
class_counts = Counter(y_train_tensor.numpy())
print(annotation_enumerated)
print("Class distribution:", class_counts)

total_samples = sum(class_counts.values())
class_weights = []
num_classes = 6

class_weights = torch.tensor([
    total_samples / (num_classes * class_counts.get(i, 1)) for i in range(num_classes)
], dtype=torch.float32).to(device)

loss = nn.CrossEntropyLoss(weight=class_weights.to(device))

#scheduler = optim.lr_scheduler.CosineAnnealingLR(adam, T_max=50)
scheduler = optim.lr_scheduler.StepLR(adam, step_size=800, gamma=0.01)


{'A': 0, 'A2': 1, 'B': 2, 'B2': 3, 'C': 4, 'C2': 5}
Class distribution: Counter({2: 220, 0: 209, 4: 60, 1: 48, 3: 47, 5: 25})


In [28]:
x_train_tensor

tensor([[-1.5227,  0.9893,  0.2192,  ...,  0.4820, -1.9236, -1.0971],
        [-1.3116, -0.2845,  0.2853,  ..., -0.0022, -0.5067, -1.3764],
        [ 0.0956,  0.5331, -0.4776,  ...,  0.0907, -0.6712, -0.6231],
        ...,
        [-0.6686, -1.7276,  1.0562,  ...,  0.5973,  0.0724,  0.1752],
        [-0.5390, -0.9106, -0.3598,  ...,  0.2842,  1.0649,  0.1721],
        [ 0.3013,  1.0376, -0.4498,  ...,  0.5709,  0.6860,  0.2495]])

In [29]:
x_train_tensor = x_train_tensor
y_train_tensor = (y_train_tensor).long()

In [30]:
print(y_train_tensor)

tensor([2, 3, 0, 2, 0, 2, 2, 0, 4, 5, 0, 4, 0, 2, 2, 2, 3, 5, 0, 2, 2, 1, 0, 2,
        2, 1, 2, 1, 4, 0, 3, 2, 0, 0, 0, 0, 0, 5, 5, 0, 0, 3, 4, 1, 1, 2, 2, 4,
        2, 0, 3, 2, 0, 0, 2, 2, 2, 2, 5, 0, 0, 0, 0, 2, 4, 2, 4, 2, 2, 0, 0, 0,
        2, 2, 1, 3, 2, 0, 1, 2, 3, 2, 2, 2, 0, 2, 2, 0, 0, 0, 5, 0, 0, 0, 0, 0,
        2, 2, 5, 0, 3, 2, 0, 1, 0, 2, 2, 0, 2, 0, 0, 0, 3, 2, 2, 0, 2, 3, 2, 2,
        0, 2, 0, 2, 0, 0, 2, 0, 2, 0, 3, 0, 0, 0, 0, 1, 2, 0, 0, 0, 4, 0, 2, 2,
        1, 2, 2, 0, 2, 2, 0, 2, 2, 5, 2, 0, 1, 2, 2, 3, 3, 2, 0, 2, 2, 2, 1, 2,
        0, 2, 4, 2, 2, 3, 0, 0, 3, 0, 2, 0, 4, 3, 4, 0, 0, 0, 2, 0, 0, 4, 2, 2,
        2, 0, 0, 0, 1, 1, 0, 2, 0, 2, 3, 2, 0, 5, 0, 2, 1, 0, 0, 2, 5, 0, 2, 2,
        0, 3, 2, 0, 2, 0, 2, 0, 0, 2, 3, 0, 0, 5, 1, 0, 0, 0, 0, 1, 0, 2, 2, 2,
        4, 0, 0, 2, 0, 1, 1, 2, 2, 1, 4, 0, 2, 4, 0, 2, 0, 3, 2, 2, 2, 0, 2, 1,
        2, 4, 2, 3, 2, 0, 3, 1, 1, 2, 4, 0, 3, 5, 0, 0, 0, 2, 0, 2, 5, 0, 0, 0,
        3, 0, 2, 0, 2, 0, 0, 2, 1, 0, 1,

In [31]:

#use TensorDatset, DataLoader from torch utils

batch_size = 8
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
#dropping last bc we are using batchnorm, so it needs >1 batch size 

for epoch in range(500):
    model.train()
    epoch_loss = 0.0

    for seqs, anns in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        seqs = seqs.to(device)
        anns = anns.to(device)
        output = model(seqs)
        output_loss = loss(output, anns)
        adam.zero_grad()
        output_loss.backward()
        adam.step()
        epoch_loss += output_loss.item() * seqs.size(0) #batch size scaling
    avg_loss = epoch_loss / len(train_dataset)
    #scheduler.step()
    print(f"Epoch {epoch+1}: Loss: {avg_loss:.4f}")





Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:02<00:00, 26.43it/s]


Epoch 1: Loss: 1.7656


Epoch 2: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:00<00:00, 271.38it/s]


Epoch 2: Loss: 1.6913


Epoch 3: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:00<00:00, 283.81it/s]


...

Epoch 356: Loss: 0.0107


Epoch 357: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 76/76 [00:00<00:00, 268.41it/s]


Epoch 357: Loss: 0.0049


Epoch 358:  79%|██████████████████████████████████████████████████████████████████████████████████████▊                       | 60/76 [00:00<00:00, 259.30it/s]


KeyboardInterrupt: 

In [32]:
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#dropping last bc we are using batchnorm, so it needs >1 batch size 

all_predictions = []
all_targets = []
def accuracy():
    model.eval()
    correct = 0
    total = 0


    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs) #model predictions [batch_size, 9]
            values, predicted = torch.max(outputs.data, 1)
            #values has highest score for each sample in batch
            #the predicted part has the classes w/ highest score for each sample
            total += targets.size(0) #add batch size
            correct += (predicted == targets).sum().item()

            #for classification report
            all_predictions.extend(predicted.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
    return 100 * correct/total

accuracy()

84.31372549019608

In [33]:

from sklearn.metrics import classification_report
import numpy as np


class_names = ['A', 'A2', 'B','B2', 'C','C2']
print(classification_report(all_targets, all_predictions, target_names=class_names))

              precision    recall  f1-score   support

           A       0.86      0.94      0.90        53
          A2       0.77      0.83      0.80        12
           B       0.92      0.84      0.88        55
          B2       0.56      0.83      0.67        12
           C       0.91      0.67      0.77        15
          C2       1.00      0.50      0.67         6

    accuracy                           0.84       153
   macro avg       0.84      0.77      0.78       153
weighted avg       0.86      0.84      0.84       153



In [34]:
import pandas as pd
import torch


ccad_filtered = pd.read_csv("clustercad_filtered.csv")
ccad_filtered_embeddings = torch.load("filtered_ccad_embeddings.pt")
ccad_filtered_embeddings = [x.mean(dim=1).squeeze(0) for x in ccad_filtered_embeddings]

'''
valid_keys = set(annotation_enumerated.keys())
is_valid = ccad_filtered["kr_type_annotation"].isin(valid_keys)


removed_indices = ccad_filtered.index[~is_valid].tolist()


ccad_filtered = ccad_filtered[is_valid].reset_index(drop=True)
ccad_filtered_embeddings = [
    emb for i, emb in enumerate(ccad_filtered_embeddings) if i not in removed_indices
]


ccad_embeddings = torch.stack(ccad_filtered_embeddings)


ccad_filtered["AnnotationEnumerated"] = ccad_filtered["kr_type_annotation"].map(annotation_enumerated)


ccad_filtered_np = ccad_filtered["AnnotationEnumerated"].to_list()
ccad_filtered_tensor = torch.tensor(ccad_filtered_np, dtype=torch.long)


print("Removed row indices:", removed_indices)
'''


'\nvalid_keys = set(annotation_enumerated.keys())\nis_valid = ccad_filtered["kr_type_annotation"].isin(valid_keys)\n\n\nremoved_indices = ccad_filtered.index[~is_valid].tolist()\n\n\nccad_filtered = ccad_filtered[is_valid].reset_index(drop=True)\nccad_filtered_embeddings = [\n    emb for i, emb in enumerate(ccad_filtered_embeddings) if i not in removed_indices\n]\n\n\nccad_embeddings = torch.stack(ccad_filtered_embeddings)\n\n\nccad_filtered["AnnotationEnumerated"] = ccad_filtered["kr_type_annotation"].map(annotation_enumerated)\n\n\nccad_filtered_np = ccad_filtered["AnnotationEnumerated"].to_list()\nccad_filtered_tensor = torch.tensor(ccad_filtered_np, dtype=torch.long)\n\n\nprint("Removed row indices:", removed_indices)\n'